Rossler performance experiments

In [1]:
import numpy as np
import torch
import sys

sys.path.append("../")
import utils as utils
import NMC as models
import importlib

## SVAM

In [2]:
# LiNGAM / SVAM performance with sparse data
import warnings

warnings.filterwarnings("ignore")
for p in [10, 50]:
    perf = []
    for i in range(20):
        # Simulate data
        T = 1000
        num_points = T
        data, GC = utils.simulate_rossler(p=p, a=0, T=T, delta_t=0.1, sd=0.05, burn_in=0, sigma=0.0)

        # format for NeuralODE
        data = torch.from_numpy(data[:, None, :].astype(np.float32))

        from benchmarks.lingam_benchmark import lingam_method

        importlib.reload(utils)
        graph = lingam_method(data.squeeze().detach())
        perf.append(utils.compare_graphs(GC, graph))  # tpr, fdr

    print("Means and standard deviations for TPR, FDR and AUC with", p, "dimensions")
    print(np.mean(np.reshape(perf, (-1, 3)), axis=0), np.std(np.reshape(perf, (-1, 3)), axis=0))

Means and standard deviations for TPR, FDR and AUC with 10 dimensions
[0.77368421 0.53953499 0.49996751] [0.10273274 0.06081473 0.02438485]
Means and standard deviations for TPR, FDR and AUC with 50 dimensions
[0.59141414 0.66847463 0.49049291] [0.10153776 0.06151317 0.02785143]


# DCM

In [3]:
# DCM performance with sparse data
for p in [10, 50]:
    perf = []
    for i in range(10):
        # Simulate data
        T = 1000
        num_points = T
        data, GC = utils.simulate_rossler(p=p, a=0, T=T, delta_t=0.1, sd=0.05, burn_in=0, sigma=0.0)

        from benchmarks.DCM import DCM_full

        graph = DCM_full(data, lambda1=0.001, s=4, w_threshold=0.1)
        # plt.matshow(abs(graph),cmap='Reds')
        # plt.colorbar()
        # plt.show()
        perf.append(utils.compare_graphs(GC, graph))  # tpr, fdr

    print("Means and standard deviations for TPR, FDR and AUC with", p, "dimensions")
    print(np.mean(np.reshape(perf, (-1, 3)), axis=0), np.std(np.reshape(perf, (-1, 3)), axis=0))

Means and standard deviations for TPR, FDR and AUC with 10 dimensions
[0.779 0.452 0.864] [0.135 0.115 0.092]
Means and standard deviations for TPR, FDR and AUC with 50 dimensions
[0.97  0.716 0.983] [2.220e-16 1.151e-01 8.450e-04]


# PCMCI

In [4]:
# pcmci performance with sparse data
for p in [10, 50]:
    perf = []
    for i in range(5):
        # Simulate data
        T = 1000
        num_points = T
        data, GC = utils.simulate_rossler(p=p, a=0, T=T, delta_t=0.1, sd=0.05, burn_in=0, sigma=0.0)

        from benchmarks.pcmci import pcmci

        importlib.reload(utils)
        graph = pcmci(data)
        perf.append(utils.compare_graphs(GC, graph))  # tpr, fdr

    print("Means and standard deviations for TPR, FDR and AUC with", p, "dimensions")
    print(np.mean(np.reshape(perf, (-1, 3)), axis=0), np.std(np.reshape(perf, (-1, 3)), axis=0))

Could not import packages for CMIknn and GPDC estimation
Could not import r-package RCIT
Means and standard deviations for TPR, FDR and AUC with 10 dimensions
[0.768 0.76  0.597] [0.123 0.041 0.083]
Means and standard deviations for TPR, FDR and AUC with 50 dimensions
[0.685 0.925 0.668] [0.068 0.006 0.018]


## NGM

In [6]:
# NGM performance with sparse data
import warnings

warnings.filterwarnings("ignore")
for p in [10, 50]:
    perf = []
    for i in range(5):
        # Simulate data
        T = 1000
        num_points = T
        data, GC = utils.simulate_rossler(p=p, a=0, T=T, delta_t=0.1, sd=0.05, burn_in=0, sigma=0.0)

        # format for NeuralODE
        data = torch.from_numpy(data[:, None, :])

        import NMC as models

        func = models.MLPODEF(dims=[p, 12, 1], GL_reg=0.1)

        # GL training
        models.train(func, data, n_steps=2000, plot=False, plot_freq=20)
        # AGL training
        # weights = func.group_weights()
        # func.GL_reg *= (1 / weights)
        # func.reset_parameters()
        # models.train(func,data,n_steps=1000,plot = True, plot_freq=20)
        graph = func.causal_graph(w_threshold=0.1)
        perf.append(utils.compare_graphs(GC, graph))  # tpr, fdr

    print("Means and standard deviations for TPR, FDR and AUC with", p, "dimensions")
    print(np.mean(np.reshape(perf, (-1, 3)), axis=0), np.std(np.reshape(perf, (-1, 3)), axis=0))

Means and standard deviations for TPR, FDR and AUC with 10 dimensions
[0.863 0.    0.932] [0.026 0.    0.013]
Means and standard deviations for TPR, FDR and AUC with 50 dimensions
[0.974 0.11  0.987] [0.005 0.043 0.002]
